# Оценка рисков по выполнению долговых обязательств

**Содержание:**  
<a id="start"></a>
1. [Импорт библиотек](#section_1)


2. [Загрузка и предобработка данных](#section_2)  
 2.1. [Разделение выборки на обучающую и тестовую](#section_2_1)  
 2.2. [Функции загрузки и предобработки данных](#section_2_2)  
 2.3. [Предобработка данных](#section_2_3)  
 
 
3. [Отбор признаков по важности (RFE)](#section_3)  


4. [Подбор гиперпараметров LightGBM](#section_4)
   

5. [Обучение модели](#section_5)

  
6. [Расчет метрик](#section_6)


7. [Submission](#section_7)  


8. [Общий вывод](#section_8)

[Страница соревнования](https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability/overview)

**Заказчик исследования:** Home Credit Bank.  
**Цель исследования:** создание модели оценки надежности заемщика.  

**Исходные данные:**  
Исходные данные представлены в датасетах:  

***Base_table***  
Базовая таблица хранит основную информацию о наблюдении и case_id. Это уникальная идентификация каждого наблюдения, она  используется для присоединения других таблиц.  

***static_0***  
Свойства: глубина=0, внутренний источник данных.  

***static_cb_0***  
Свойства: глубина=0, внешний источник данных.  

***applprev_1***  
Свойства: глубина=1, внутренний источник данных.  

***other_1***  
Свойства: глубина=1, внутренний источник данных.  

***tax_registry_a_1***  
Свойства: глубина=1, внешний источник данных, поставщик налогового реестра A.  

***tax_registry_b_1***  
Свойства: глубина=1, внешний источник данных, поставщик налогового реестра B.  

***tax_registry_c_1***  
Свойства: глубина=1, внешний источник данных, поставщик налогового реестра C.  

***credit_bureau_a_1***  
Свойства: глубина=1, внешний источник данных, поставщик кредитного бюро A.  

***credit_bureau_b_1***  
Свойства: глубина=1, внешний источник данных, поставщик кредитного бюро B.  

***deposit_1***  
Свойства: глубина=1, внутренний источник данных.  

***person_1***  
Свойства: глубина=1, внутренний источник данных.  

***debitcard_1***  
Свойства: глубина=1, внутренний источник данных.  

***applprev_2***  
Свойства: глубина=2, внутренний источник данных.  

***person_2***  
Свойства: глубина=2, внутренний источник данных.  

***credit_bureau_a_2***  
Свойства: глубина=2, внешний источник данных, поставщик кредитного бюро A.  

***credit_bureau_b_2***  
Свойства: глубина=2, внешний источник данных, поставщик кредитного бюро B.  


глубина=0 — это статические функции, напрямую привязанные к определенному case_id.  
глубина=1 — каждый case_id имеет связанную историческую запись, проиндексированную по num_group1.  
глубина=2 — каждый case_id имеет связанную историческую запись, проиндексированную по num_group1 и num_group2.  

**Описание признаков**

- case_id - Это уникальный идентификатор для каждого кредитного дела. По этому признаку производится объединение таблиц;  
- date_decision - дата принятия решения об одобрении кредита;  
- WEEK_NUM - номер недели принятия решения об одобрении кредита, используемый для агрегации;  
- MONTH - месяц принятия решения об одобрении кредита, используемый для агрегации;  
- target - целевой признак. Определяется по истечении определенного периода времени в зависимости от того, допустил ли клиент дефолт по конкретному кредитному делу (займу);  
- num_group1 - индексация, используемая для исторических записей case_id в таблицах глубины 1 и глубины 2;  
- num_group2 - индексация исторических записей case_id таблиц глубины 2.  

Все остальные необработанные столбцы в таблицах служат предикторами.  
Для таблиц с глубиной 0 предикторы могут напрямую использоваться как признаки. Однако для таблиц с глубиной 1 и 2 может потребоваться использовать функции агрегации, которые сожмут исторические записи, связанные с каждым case_id в один признак.
В случае, если num_group1 или num_group2 обозначает индекс лица (это ясно из определений предиктора), нулевой индекс имеет особое значение. Когда num_groupN равен 0, это заявитель (лицо, которое обратилось за кредитом).  

Признаки разделены на группы:  
P - Transform DPD (Дни просрочки)  
M - Маскировка категорий  
A - Преобразовать сумму  
D - Дата преобразования  
T — Неопределенное преобразование  
L — Неопределенное преобразование  
Принадлежность к группе определяется заглавной буквой в конце имени предиктора (например, maxdbddpdtollast6m_4187119P).  

[Подробная информация по данным](https://www.kaggle.com/competitions/home-credit-credit-risk-model-stability/data)

  
**Целевая метрика:**  
Целевая метрика - stability metric:  
stability metric = mean(gini) + 88,0 ⋅ min(0, a) − 0,5 ⋅ std(residuals)  

Показатель gini рассчитывается для прогнозов, соответствующих каждому WEEK_NUM:  
gini = 2 ⋅ AUC − 1  

Линейная регрессия, а ⋅ х + b, проходит через недельные показатели gini, a falling_rate рассчитывается как min(0, а).  

Изменчивость прогнозов рассчитывается путем взятия стандартного отклонения остатков из приведенной выше линейной регрессии с применением штрафа к изменчивости модели.

## Импорт библиотек
<a id="section_1"></a>

In [1]:
import gc
import numpy as np
import os
import pandas as pd
import re

import lightgbm as lgb
import phik
import polars as pl

from catboost import CatBoostClassifier, Pool
import category_encoders as ce
from glob import glob
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from pathlib import Path
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder

[К содержанию](#start)

## Загрузка и предобработка данных
<a id="section_2"></a>

### Разделение выборки на обучающую и тестовую
<a id="#section_2_1"></a>

Задаем константы:

In [2]:
RS=42                                                # random_state
NUM_SAMPLE_TRAIN = 100000                            # количество наблюдений в обучающей выборке
NUM_SAMPLE_TEST = int(NUM_SAMPLE_TRAIN*0.25)         # количество наблюдений в тестовой выборке

Задаем пути к исходным данным:

In [3]:
ROOT = Path('/kaggle/input/home-credit-credit-risk-model-stability')                   # путь kaggle
if os.path.exists(ROOT) == False:
    ROOT = Path('D:/ds/Jupiter/Project_data_sciense/kaggle/HomeCredit/data/')          # путь локальный

TRAIN_DIR       = ROOT / 'parquet_files' / 'train'
TEST_DIR        = ROOT / 'parquet_files' / 'test'

Разделение данных производим следующим образом: разделяем данные на тестовую выборку и выборку, предназначенную для предобработки данных. Из последней после предобработки данных будет выделена обучающая выборка. Таким образом, при предобработке будут максимально учтены взаимосвязи признаков, а обучение по времени уложится в заданные организаторами соревнования лимиты (не более 12 часов).

In [4]:
data = pl.read_parquet(TRAIN_DIR / 'train_base.parquet')                                # загрузка основного датафрейма
data = data.with_columns((pl.col('target').cast(pl.String) + pl.col('WEEK_NUM')         # создание признака для стратификации                     
                          .cast(pl.String)).alias('stratify'))
TRAIN_SIZE = NUM_SAMPLE_TRAIN / data.shape[0]                                           # доля данных для обучающей выборки
TEST_SIZE = NUM_SAMPLE_TEST / data.shape[0]                                             # доля данных для тестовой выборки
# разделение данных на тестовую выборку и выборку для предобработки данных
X_preprocessing, X_test, y_preprocessing, y_test = train_test_split(
    data,
    data.select(pl.col('target')),
    stratify=data.select(pl.col('stratify')),
    test_size=TEST_SIZE,
    random_state=RS)
# выделяем данные для стратификации (будут использованы при выделении из X_preprocessing обучающей выборки)
data_stratify = X_preprocessing['stratify']
weeks_stratify = X_preprocessing['WEEK_NUM']
# список case_id обучающей выборки
id_preprocessing_list = X_preprocessing.sort(pl.col('case_id'))['case_id'].explode().unique().to_list()
# список case_id тестовой выборки
id_test_list = X_test.sort(pl.col('case_id'))['case_id'].explode().unique().to_list()
del data

### Функции загрузки и предобработки данных
<a id="#section_2_2"></a>

In [5]:
class Pipeline:
    '''Класс предобработки данных'''
    
    def fill_null_str(df):
        '''Функция обработки пропусков строковых признаков'''
        
        for col in df.columns:                                                            # проходим по названиям признаков
            # если признак категориальный или булевый
            if (pd.api.types.is_numeric_dtype(df[col]) != True) | (pd.api.types.is_bool_dtype(df[col]) == True):
                df = df.astype({col : 'category'})                                        # изменяем тип на категориальный
        return df

    def set_table_dtypes(df):
        '''Функция изменения типа данных'''
        
        for col in df.columns:
            # если признак - case_id, WEEK_NUM, num_group1, num_group2 или признак, содержащий количество дней
            if (col in ['case_id', 'WEEK_NUM', 'num_group1', 'num_group2']) | (col[-1] in ('P',)):              
                df = df.with_columns(pl.col(col).cast(pl.Int64))                          # изменяем тип на целочисленный
            elif col[-1] in ('A',):                                                       # если признак - сумма средств                                                   
                df = df.with_columns(pl.col(col).cast(pl.Float64))                        # изменяем тип на float
            elif col[-1] in ('M',):                                                       # если признак - закодированные текстовые значения 
                df = df.with_columns(pl.col(col).cast(pl.String))                         # изменяем тип на строковый
            elif (col[-1] in ('D',)) | (col in ['date_decision']):                        # если признак - дата
                df = df.with_columns(pl.col(col).cast(pl.Date))                           # изменяем тип на Date
        return df

    def handle_dates(df):
        '''Функция получения количества дней из признаков типа Date'''
        
        for col in df.columns:
            if col[-1] in ('D',):                                                          # если признак - дата
                df = df.with_columns(pl.col(col) - pl.col('date_decision'))                # из признака вычитаем date_decision
                df = df.with_columns(pl.col(col).dt.total_days())                          # получаем количество дней
        df = df.drop('date_decision', 'MONTH')                                             # удаляем признаки date_decision и MONTH
        return df

    def filter_cols(df):
        for col in df.columns:
            if col not in ["target", "case_id", "WEEK_NUM"]:
                isnull = df[col].is_null().mean()
                if isnull > 0.7:
                    df = df.drop(col)
        
        for col in df.columns:
            if (col not in ["target", "case_id", "WEEK_NUM"]) & (df[col].dtype == pl.String):
                freq = df[col].n_unique()
                if (freq == 1) | (freq > 200):
                    df = df.drop(col)
        
        return df

In [6]:
class Aggregator:
    '''Класс создание агрегированных признаков'''
    
    def num_expr(df):
        '''Агрегация числовых признаков (количество дней и средств)'''
        
        cols = [col for col in df.columns if col[-1] in ("P", "A")]                # получаем список признаков
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]               # агрегируем максимальные значения
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]            # агрегируем последние значения
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]            # агрегируем средние значения
        return expr_max +expr_last+expr_mean
    
    def date_expr(df):
        '''Агрегация признаков дат'''
        
        cols = [col for col in df.columns if col[-1] in ("D")]                     # получаем список признаков
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]               # агрегируем максимальные значения
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]            # агрегируем последние значения
        expr_mean = [pl.mean(col).alias(f"mean_{col}") for col in cols]            # агрегируем средние значения
        return  expr_max +expr_last+expr_mean
    
    def str_expr(df):
        '''Агрегация строковых признаков'''
        
        cols = [col for col in df.columns if col[-1] in ("M",)]                    # получаем список признаков
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]               # агрегируем максимальное значения
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]            # агрегируем последние значения
        return  expr_max +expr_last
    
    def other_expr(df):
        '''Агрегация прочих признаков'''
        
        cols = [col for col in df.columns if col[-1] in ("T", "L")]                # получаем список признаков
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]               # агрегируем максимальные значения
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]            # агрегируем последние значения
        return  expr_max +expr_last
    
    def count_expr(df):
        '''Агрегация признаков num_group1/num_group2'''
        
        cols = [col for col in df.columns if "num_group" in col]                   # получаем список признаков
        expr_max = [pl.max(col).alias(f"max_{col}") for col in cols]               # агрегируем максимальные значения
        expr_last = [pl.last(col).alias(f"last_{col}") for col in cols]            # агрегируем последние значения
        return  expr_max +expr_last
    
    def get_exprs(df):
        '''Функция создания датафрейма с агрегированными признаками'''
        
        exprs = Aggregator.num_expr(df) + \
                Aggregator.date_expr(df) + \
                Aggregator.str_expr(df) + \
                Aggregator.other_expr(df) + \
                Aggregator.count_expr(df)

        return exprs

In [7]:
class ReadData:
    '''Класс чтения данных'''
    
    def read_file(path, id_ind=[], depth=None):
        '''Чтение данных, содержащихся в единственном файле'''

        df = pl.read_parquet(path)                                         # читаем файл
        df = df.pipe(Pipeline.set_table_dtypes)                            # преобразовываем типы данных
        if id_ind != []:                                                   # если список case_id не пуст
            df = df.filter(pl.col('case_id').is_in(id_ind))                # выбираем наблюдения с значениями case_id , указанными в списке
        if depth in [1,2]:                                                 # если в словаре датафрейм находится на глубине 1 или 2
            df = df.group_by('case_id').agg(Aggregator.get_exprs(df))      # отправляем датафрейм на агрегацию признаков
        return df

    def read_files(regex_path, id_ind=[], depth=None):
        '''Чтение данных, содержащихся в нескольких файлах файле'''
        
        chunks = []                                                        # инициализируем пустой список
    
        for path in glob(str(regex_path)):                                 # перебираем пути к файлам
            df = pl.read_parquet(path)                                     # читаем файл
            df = df.pipe(Pipeline.set_table_dtypes)                        # преобразовываем типы данных
            if id_ind != []:                                               # если список case_id не пуст
                df = df.filter(pl.col('case_id').is_in(id_ind))            # выбираем наблюдения с значениями case_id , указанными в списке
            if depth in [1,2]:                                             # если в словаре датафрейм находится на глубине 1 или 2
                df = df.group_by('case_id').agg(Aggregator.get_exprs(df))  # отправляем датафрейм на агрегацию признаков
            chunks.append(df)                                              # добавляем датафрейм в список
    
        df = pl.concat(chunks, how='vertical_relaxed')                     # конкатенируем датафреймы списка
        if id_ind == []:                                                   # если список case_id пуст
            df = df.unique(subset=['case_id'])                             # получаем уникальные case_id
        return df

In [8]:
class JoinDataFrame:
    '''Класс объединения датафреймов в один'''

    def feature_eng(df_base, depth_0, depth_1, depth_2):
        '''Функция объединения данных в датафрейм, подготовленный для создания агрегированных признаков'''
        df_base = (                                                              # добавляем признаки - месяц и день недели
            df_base
            .with_columns(
                month_decision = pl.col('date_decision').dt.month(),
                weekday_decision = pl.col('date_decision').dt.weekday(),
            )
        )
        # проходим по датафреймам находящимся на глубине 0-2 словаря датафреймов
        for i, df in enumerate(depth_0 + depth_1 + depth_2):
            df_base = df_base.join(df, how='left', on='case_id', suffix=f'_{i}')   # присоединяем датафреймы к базовому по case_id
        df_base = df_base.pipe(Pipeline.handle_dates)                              # из дат получаем количество дней
        return df_base

In [9]:
def reduce_mem_usage(df):
    '''Функция оптимизации по используемой памяти'''
    
    start_mem = df.memory_usage().sum() / 1024**2                                # начальное значение используемой памяти
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:                                                       # перебираем признаки
        col_type = df[col].dtype                                                 # получаем тип данных признака
        if str(col_type)=='category':                                            # если тип признака - category
            continue                                                             # продолжаем
        
        if (col_type != object) & (str(col_type) !='bool'):                      # если тип признака не object и не bool
            c_min = df[col].min()                                                # получаем минимальное значение признака
            c_max = df[col].max()                                                # получаем максимальное значение признака
            # если тип данных - целочисленные
            if str(col_type)[:3] == 'int':
                # проверяем в какой диапазон попадают значения и назначаем соответствующий тип
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            # если тип данных - не целочисленные
            else:
                # проверяем в какой диапазон попадают значения и назначаем соответствующий тип
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            continue
    end_mem = df.memory_usage().sum() / 1024**2                                  # конечное значение используемой памяти
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [10]:
def read_file_dict(data_type='train', id_ind=[]):
    '''Функция загрузки датафреймов в словарь
       Параметр 'data_type' - тип загружаемых данных (обучающая или тестовая)
       Параметр 'id_ind' - список case_id, на которых будет основана обучающая выборка'''
    
    # указываем путь до конечной папки
    if data_type == 'train':
        path = TRAIN_DIR
    else:
        path = TEST_DIR
    # загрузка датафреймов в словарь
    data_store = {
        'df_base': ReadData.read_file(path / str(data_type+'_base.parquet'), id_ind=id_ind),
        'depth_0': [
            ReadData.read_file(path / str(data_type+'_static_cb_0.parquet'), id_ind=id_ind),
            ReadData.read_files(path / str(data_type+'_static_0_*.parquet'), id_ind=id_ind),
        ],
        'depth_1': [
            ReadData.read_files(path / str(data_type+'_applprev_1_*.parquet'), id_ind=id_ind, depth=1),
            ReadData.read_file(path / str(data_type+'_tax_registry_a_1.parquet'), id_ind=id_ind, depth=1),
            ReadData.read_file(path / str(data_type+'_tax_registry_b_1.parquet'), id_ind=id_ind, depth=1),
            ReadData.read_file(path / str(data_type+'_tax_registry_c_1.parquet'), id_ind=id_ind, depth=1),
            ReadData.read_files(path / str(data_type+'_credit_bureau_a_1_*.parquet'), id_ind=id_ind, depth=1),
            ReadData.read_file(path / str(data_type+'_credit_bureau_b_1.parquet'), id_ind=id_ind, depth=1),
            ReadData.read_file(path / str(data_type+'_other_1.parquet'), id_ind=id_ind, depth=1),
            ReadData.read_file(path / str(data_type+'_person_1.parquet'), id_ind=id_ind, depth=1),
            ReadData.read_file(path / str(data_type+'_deposit_1.parquet'), id_ind=id_ind, depth=1),
            ReadData.read_file(path / str(data_type+'_debitcard_1.parquet'), id_ind=id_ind, depth=1),
        ],
        'depth_2': [
            ReadData.read_file(path / str(data_type+'_credit_bureau_b_2.parquet'), id_ind=id_ind, depth=2),
            ReadData.read_files(path / str(data_type+'_credit_bureau_a_2_*.parquet'), id_ind=id_ind, depth=2),
            ReadData.read_file(path / str(data_type+'_applprev_2.parquet'), id_ind=id_ind, depth=2),
            ReadData.read_file(path / str(data_type+'_person_2.parquet'), id_ind=id_ind, depth=2)
        ]
    }
    return data_store

In [11]:
def data_preprocessing(data_store):
    '''Функция преобразования обучающих данных.
       'data_store' - словарь датафреймов.'''
    
    # объединяем данные в датафрейм, подготовленный для агрегации признаков                                             
    df = JoinDataFrame.feature_eng(**data_store)  
    df = df.pipe(Pipeline.filter_cols)                                # отсеиваем малоинформативные признаки
    df = df.to_pandas()                                               # преобразовываем датафрейм из polars в pandas
    df = df.pipe(Pipeline.fill_null_str)
    df = reduce_mem_usage(df)                                         # оптимизируем датафрейм по памяти
    return df

In [12]:
class CorrFilter:
    '''Класс фильтрации признаков по степени корреляции'''
    
    def reduce_group(df, list_features):
        '''функция получения списка признаков'''
        
        use = []                                                    # инициализируем список используемых признаков
        # перебираем список оставшихся после фильтрации по порогу корреляции признаков (получаем подсписки)
        for sublist in list_features:
            sublist_len = 0                                         # инициализируем переменную (длина подсписка)
            value_current = sublist[0]                              # получаем первое значение подсписка
            for value in sublist:                                   # проходим по значениям подсписка
                sublist_len_current = df[value].nunique()           # количество уникальных значений признака
                if sublist_len_current > sublist_len:               # если количество уникальных значений признака больше текущего значения переменной
                    sublist_len = sublist_len_current               # переменной присваивается значение количества уникальных значений признака
                    value_current = value                           # обновляем значение текущего признака
            use.append(value_current)                               # добавляем текущее значение признака в список используемых признаков
        print('Use these',use)
        return use
    
    
    def cat_columns_by_correlation(matrix, threshold):
        '''функция фильтрации категориальных признаков'''
        
        correlation_matrix = matrix.phik_matrix().values           # рассчитываем матрицу корреляций
        remaining_cols = list(matrix.phik_matrix().columns)  
        correlation_matrix = pd.DataFrame(data=correlation_matrix, columns=remaining_cols, index=remaining_cols)
        groups = []                                                # инициализируем список                    # список признаков
        while remaining_cols:                                      # перебираем список, пока в нем есть значения
            col = remaining_cols.pop(0)                            # выбираем первое значение списка, удаляя его из списка
            group = [col]                                          # инициализируем список текущим признаком
            for c in remaining_cols:                               # перебираем значения списка признаков
                if correlation_matrix.loc[col, c] >= threshold:    # если корреляция признаков превышает или равен пороговому значению
                    group.append(c)                                # добавляем текущий признак в список
            groups.append(group)                                   # собираем списки признаков в общий список
            remaining_cols = [c for c in remaining_cols if c not in group] # из списка списков создаем список признаков
        return groups
    
    def num_columns_by_correlation(matrix, threshold):
        '''функция фильтрации числовых признаков'''
        
        correlation_matrix = matrix.corr()                         # рассчитываем матрицу корреляций
        groups = []                                                # инициализируем список
        remaining_cols = list(matrix.columns)                      # список признаков
        while remaining_cols:                                      # перебираем список, пока в нем есть значения
            col = remaining_cols.pop(0)                            # выбираем первое значение списка, удаляя его из списка
            group = [col]                                          # инициализируем список текущим признаком
            for c in remaining_cols:                               # перебираем значения списка признаков
                if correlation_matrix.loc[col, c] >= threshold:    # если корреляция признаков превышает или равен пороговому значению
                    group.append(c)                                # добавляем текущий признак в список
            groups.append(group)                                   # собираем списки признаков в общий список
            remaining_cols = [c for c in remaining_cols if c not in group]  # из списка списков создаем список признаков
        return groups

In [13]:
def cat_num_separae(df):
    '''функция создания списков количественных и категориальных признаков'''
    
    num_cols = []                                                           # инициализация списка числовых признаков
    cat_cols = df.select_dtypes(include=['category']).columns.to_list()     # список категориальных признаков
    for i in df.columns:                                                    # цикл по признакам
        if i not in cat_cols:                                               # если признак не категориальный
            num_cols.append(i)                                              # добавляем признак в список количественных
    return num_cols, cat_cols

In [14]:
def columns_sync(df, union_features_list):
    '''функция синхронизации перечня и типа признаков тестовой выборки с обучающей'''
    
    df_standard = df_preprocessing.copy(deep=True)                                  # создаем копию обучающей выборки
    df_standard['case_id'] = df_preprocessing_sync['case_id']                       # добавляем признак case_id в эталонную выборку
    list_test_id = df['case_id'].to_list()                                  # список case_id тестовой выборки
    df_united = pd.concat([df_standard, df], ignore_index=True)             # объединение эталонной и тестовой выборок
    df = df_united.loc[df_united['case_id'].isin(list_test_id)]             # выделяем тестовые наблюдения
    df = df[union_features_list].reset_index(drop=True)                     # оставляем только признаки, входящие в эталонную выборку
    return df

In [15]:
def cat_fillna(df):
    '''функция заполнения пропусков категориальных признаков'''
    
    for col in cat_cols:
        df = df.astype({col : 'object'})                          # изменяем тип на строковый
        df[col] = df[col].fillna('unknown').infer_objects(copy=False)
        df = df.astype({col : 'category'})                         # изменяем тип на категориальный
        df[col] = df[col].cat.remove_unused_categories()
    return df

In [16]:
def one_unique_target(df):
    '''функция удаления из выборки наблюдений с теми значениями WEEK_NUM для которых присутствует только одно значение таргет'''
    
    target_nunique = df.groupby('WEEK_NUM')['target'].nunique().reset_index()
    target_nunique_drop_lost = (target_nunique.loc[target_nunique['target'] == 1]['WEEK_NUM']).to_list()
    df = df.loc[~df['WEEK_NUM'].isin(target_nunique_drop_lost)].reset_index(drop=True)
    return df

In [17]:
def col_number(df):
    '''функция подсчета количества признаков'''
    
    print('Количество категориальных признаков: {}'.format(len(cat_cols)))
    print('категориальные признаки: {}'.format(cat_cols))
    print()
    print('Количество числовых признаков: {}'.format(len(num_cols)))
    print('Числовые признаки: {}'.format(num_cols))
    print()
    print('Общее количество признаков: {}'.format(df.shape[1]))

### Предобработка данных
<a id="#section_2_3"></a>

Предобработка данных включает в себя следующие этапы:
- получение агрегированных признаков (min, max, mean, first, last) для каждого значения case_id;
- заполнение пропусков нечисловых признаков значением 'unknown';
- преобразование типов данных;
- генерация новых признаков из дат;
- фильтрация признаков по порогу количества пропущенных значений и количеству уникальных значений;
- фильтрация признаков по пороговому значению корреляции.

**Обучающая выборка**

Предобрабатывается вся исходная выборка за исключением элементов, выделенных в тестовую подвыборку, дабы максимально точно определить признаки с высокой степенью корреляции. После предобработки выделим обучающую выборку в количестве наблюдений, соответствующем значению NUM_SAMPLE_TRAIN.

Загружаем датафреймы в словарь, осуществляем предобработку данных, агрегируем данные и объединяем их в один датафрейм:

In [18]:
data_store = read_file_dict(id_ind=id_preprocessing_list)                                # получаем словарь датафреймов
# предобработка данных - получаем объединенный датафрейм и список рассматриваемых case_id
df_preprocessing = data_preprocessing(data_store)
del data_store                                               # удаляем список из памяти

Memory usage of dataframe is 4251.96 MB
Memory usage after optimization is: 1502.35 MB
Decreased by 64.7%


Выделяем целевую переменную:

In [19]:
y_preprocessing = df_preprocessing['target']
# weeks = df_preprocessing['WEEK_NUM']
df_preprocessing_sync = df_preprocessing[['case_id', 'WEEK_NUM', 'target']]
df_preprocessing = df_preprocessing.drop(['case_id', 'WEEK_NUM', 'target'], axis=1)

Создаем списки числовых и категориальных признаков:

In [20]:
num_cols, cat_cols = cat_num_separae(df_preprocessing)

Заполняем пропущенные значения категориальных признаков на 'unknown':

In [21]:
df_preprocessing = cat_fillna(df_preprocessing)

C:\Users\bond_\AppData\Local\Temp\ipykernel_31932\3227710926.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna('unknown').infer_objects(copy=False)


Задаем порог уровня корреляции:

In [22]:
THRESHOLD=0.9

Отсеиваем числовые признаки по порогу корреляции:

In [23]:
num_corr = CorrFilter.num_columns_by_correlation(df_preprocessing[num_cols], THRESHOLD)
num_feature_list = CorrFilter.reduce_group(df_preprocessing[num_cols], num_corr)

Use these ['month_decision', 'weekday_decision', 'max_birth_259D', 'days180_256L', 'days30_165L', 'days360_512L', 'firstquarter_103L', 'fourthquarter_440L', 'pmtscount_423L', 'pmtssum_45A', 'responsedate_1012D', 'responsedate_4527233D', 'secondquarter_766L', 'thirdquarter_1082L', 'max_actualdpd_943P', 'amtinstpaidbefduel24m_4187115A', 'annuity_780A', 'annuitynextmonth_57A', 'applicationcnt_361L', 'applications30d_658L', 'applicationscnt_1086L', 'applicationscnt_464L', 'applicationscnt_867L', 'mindbddpdlast24m_3658935P', 'avginstallast24m_3658937A', 'avgmaxdpdlast9m_3716943P', 'avgoutstandbalancel6m_4187114A', 'avgpmtlast12m_4525200A', 'clientscnt_1022L', 'clientscnt_100L', 'clientscnt_1071L', 'clientscnt_1130L', 'clientscnt_157L', 'clientscnt_257L', 'clientscnt_304L', 'clientscnt_360L', 'clientscnt_493L', 'clientscnt_533L', 'clientscnt_887L', 'clientscnt_946L', 'cntincpaycont9m_3716944L', 'cntpmts24_3658933L', 'commnoinclast6m_3546845L', 'disbursedcredamount_1113A', 'currdebtcredtypera

Определяем количество признаков:

In [24]:
union_features_list = num_feature_list + cat_cols           # Объединяем списки числовых и категориальных признаков
df_preprocessing = df_preprocessing[union_features_list]            # Оставляем в датафрейме отфильтрованные признаки
num_cols, cat_cols = cat_num_separae(df_preprocessing)              # Обновляем списки числовых и категориальных признаков
col_number(df_preprocessing)                                        # Определяем количество признаков

Количество категориальных признаков: 114
категориальные признаки: ['description_5085714M', 'education_1103M', 'education_88M', 'maritalst_385M', 'maritalst_893M', 'requesttype_4525192L', 'credtype_322L', 'disbursementtype_67L', 'inittransactioncode_186L', 'isbidproduct_1095L', 'lastapprcommoditycat_1041M', 'lastcancelreason_561M', 'lastrejectcommoditycat_161M', 'lastrejectcommodtypec_5251769M', 'lastrejectreason_759M', 'lastrejectreasonclient_4145040M', 'lastst_736L', 'opencred_647L', 'paytype1st_925L', 'paytype_783L', 'twobodfilling_608L', 'max_cancelreason_3545846M', 'max_education_1138M', 'max_postype_4733339M', 'max_rejectreason_755M', 'max_rejectreasonclient_4145042M', 'last_cancelreason_3545846M', 'last_education_1138M', 'last_postype_4733339M', 'last_rejectreason_755M', 'last_rejectreasonclient_4145042M', 'max_credtype_587L', 'max_familystate_726L', 'max_inittransactioncode_279L', 'max_isbidproduct_390L', 'max_status_219L', 'last_credtype_587L', 'last_familystate_726L', 'last_in

Подготавливаем датафрейм к выделению обучающей подвыборки: добавляем необходимые признаки.

In [25]:
data = df_preprocessing.copy(deep=True)                                      # копия датафрейма
data['stratify'] = data_stratify                                             # признак, по которому производится стратификация
# добавляем признаки 'case_id', 'WEEK_NUM', 'target'
data[['case_id', 'WEEK_NUM', 'target']] = df_preprocessing_sync[['case_id', 'WEEK_NUM', 'target']]

Выделяем обучающую выборку:

In [26]:
df_train, df_drop, y, y_drop = train_test_split(
    data,
    y_preprocessing,
    stratify=data_stratify,
    train_size=TRAIN_SIZE,
    random_state=RS)

Поскольку модель прогнозирования основывается на группировании наблюдений по признаку WEEK_NUM, удаляем из выборки признаки с теми значениями WEEK_NUM для которых присутствует только одно значение таргет:

In [27]:
df_train['target'] = y                            # добавляем целевой признак
#Поскольку модель прогнозирования основывается на группировании наблюдений по признаку WEEK_NUM, удаляем из выборки признаки
#с теми значениями WEEK_NUM для которых присутствует только одно значение таргет:
df_train = one_unique_target(df_train)
y = df_train['target']                            # выделяем целевой признак

Выделяем признак WEEK_NUM, по которому будет производится группировка наблюдений при обучении:

In [28]:
weeks = df_train['WEEK_NUM']
df_train_sync = df_train[['case_id', 'WEEK_NUM', 'target']]                        # данные для расчета итоговой метрики
df_train = df_train.drop(['case_id', 'WEEK_NUM', 'target', 'stratify'], axis=1)

**Тестовая выборка**

Загружаем датафреймы в словарь, осуществляем предобработку данных, агрегируем данные и объединяем их в один датафрейм:

In [29]:
data_store = read_file_dict(id_ind=id_test_list)   

In [30]:
data_store = read_file_dict(id_ind=id_test_list)                                # получаем словарь датафреймов
# предобработка данных - получаем объединенный датафрейм и список рассматриваемых case_id
df_test = data_preprocessing(data_store)
del data_store                                                                   # удаляем список из памяти

Memory usage of dataframe is 72.39 MB
Memory usage after optimization is: 25.00 MB
Decreased by 65.5%


Поскольку модель прогнозирования основывается на группировании наблюдений по признаку WEEK_NUM, удаляем из выборки признаки с теми значениями WEEK_NUM для которых присутствует только одно значение таргет:

In [31]:
df_test = one_unique_target(df_test)

Выделяем целевую переменную и признак, WEEK_NUM, по которому будет производится группировка наблюдений при проверке модели:

In [32]:
y_test = df_test['target']
weeks_test = df_test['WEEK_NUM']
df_test_sync = df_test[['case_id', 'WEEK_NUM', 'target']]

Синхронизируем набор признаков с в обучающей выборкой:

In [33]:
df_test = columns_sync(df_test, union_features_list)

Заполняем пропущенные значения категориальных признаков на 'unknown':

In [34]:
df_test = cat_fillna(df_test)

C:\Users\bond_\AppData\Local\Temp\ipykernel_31932\3227710926.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna('unknown').infer_objects(copy=False)
C:\Users\bond_\AppData\Local\Temp\ipykernel_31932\3227710926.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna('unknown').infer_objects(copy=False)


**Тестовая выборка (submission)**

Загружаем датафреймы в словарь, осуществляем предобработку данных, агрегируем данные и объединяем их в один датафрейм:

In [35]:
data_store = read_file_dict(data_type='test')
df_test_kaggle = data_preprocessing(data_store)
del data_store

Memory usage of dataframe is 0.04 MB
Memory usage after optimization is: 0.02 MB
Decreased by 45.3%


Выделяем целевую переменную и признак, WEEK_NUM, по которому будет производится группировка наблюдений при проверке модели:

In [36]:
df_test_kaggle_sync = df_test_kaggle[['case_id', 'WEEK_NUM']]

Синхронизируем набор признаков с в обучающей выборкой:

In [37]:
df_test_kaggle = columns_sync(df_test_kaggle, union_features_list)

Заполняем пропущенные значения категориальных признаков на 'unknown':

In [38]:
df_test_kaggle = cat_fillna(df_test_kaggle)

C:\Users\bond_\AppData\Local\Temp\ipykernel_31932\3227710926.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna('unknown').infer_objects(copy=False)
C:\Users\bond_\AppData\Local\Temp\ipykernel_31932\3227710926.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df[col] = df[col].fillna('unknown').infer_objects(copy=False)


***Вывод:***  
1. Получены 3 выборки:
   - обучающая - df_preprocessing;
   - тестовая - df_test;
   - тестовая для выгрузки в kaggle -df_test_kaggle.

2. Выполнена следующая предобработка:
   - получение агрегированных признаков (min, max, mean, first, last) для каждого значения case_id;
   - заполнение пропусков нечисловых признаков значением 'unknown';
   - преобразование типов данных;
   - генерация новых признаков из дат;
   - фильтрация признаков по порогу количества пропущенных значений и количеству уникальных значений;
   - фильтрация признаков по пороговому значению корреляции.

3. По результатам предобработки выборки имеют следующее количество признаков:
   - количество категориальных признаков: 38;
   - количество числовых признаков: 137;
   - общее количество признаков: 175.

[К содержанию](#start)

## Отбор признаков по важности (RFE)
<a id="#section_3"></a>

Если количество признаков велико, целесообразно его (количество) сократить, чтобы уменьшить время обучения модели. Ставим максимальное количество признаков - 300:

In [39]:
def rfe_calc(X_train, y_train):
    '''функция отбора признаков по важности'''
    
    X_rfe = X_train.copy(deep=True)                                           # копия обучающей выборки
    encoder = ce.CatBoostEncoder()                                            # инициализация кодировщика
    encoder.fit(X_rfe[cat_cols], y_train)                                     # обучение кодировщика
    X_rfe[cat_cols] = encoder.transform(X_rfe[cat_cols])                      # кодировка категориальных признаков
    X = X_rfe.values                                                          # значения входных признаков
    Y = y_train.values                                                        # значения целевого признака
    model = CatBoostClassifier(task_type='GPU',                               # инициализация модели catboost
                               verbose=1000,
                               random_state=RS)
    rfe = RFE(model, n_features_to_select=300, step=3)                              # инициализация модели rfe
    fit = rfe.fit(X, Y)                                                             # обучение модели rfe
    df_feature = pd.DataFrame(rfe.support_, index=X_rfe.columns, columns=['Rank'])  # объединение признаков со значениями значимости в датафрейм
    df_feature = df_feature.query('Rank == True')                                   # выделяем признаки, проходящих по порогу значимости
    list_feature = df_feature.index                                                 # перечень отобранных признаков
    list_feature = list_feature.to_list()                                           # список отобранных признаков
    return list_feature

In [40]:
if df_train.shape[1] > 300:                                                       # если количество признаков превышает порог
    list_feature = rfe_calc(df_train, y)
    df_train = df_train[list_feature]
    df_test = df_test[list_feature]
    df_test_kaggle = df_test_kaggle[list_feature]
    num_cols, cat_cols = cat_num_separae(df_train)
    col_number(df_train)
else:
    col_number(df_train)

Learning rate set to 0.026193
0:	learn: 0.6496764	total: 165ms	remaining: 2m 44s
999:	learn: 0.0995325	total: 18.3s	remaining: 0us
Learning rate set to 0.026193
0:	learn: 0.6490167	total: 9.76ms	remaining: 9.75s
999:	learn: 0.0997412	total: 9.33s	remaining: 0us
Learning rate set to 0.026193
0:	learn: 0.6486813	total: 10.6ms	remaining: 10.6s
999:	learn: 0.0996906	total: 1m 49s	remaining: 0us
Learning rate set to 0.026193
0:	learn: 0.6499440	total: 9.55ms	remaining: 9.54s
999:	learn: 0.0996708	total: 9.16s	remaining: 0us
Learning rate set to 0.026193
0:	learn: 0.6495327	total: 9.75ms	remaining: 9.74s
999:	learn: 0.0997250	total: 8.61s	remaining: 0us
Learning rate set to 0.026193
0:	learn: 0.6489096	total: 12.4ms	remaining: 12.4s
999:	learn: 0.0996480	total: 8.53s	remaining: 0us
Learning rate set to 0.026193
0:	learn: 0.6490747	total: 9.58ms	remaining: 9.57s
999:	learn: 0.0996277	total: 8.45s	remaining: 0us
Learning rate set to 0.026193
0:	learn: 0.6496030	total: 9.51ms	remaining: 9.5s
99

[К содержанию](#start)

## Подбор гиперпараметров LightGBM
<a id="#section_4"></a>

Обучатся будут две модели - CatBoostClassifier и LGBMClassifier. Время подбора параметров для CatBoostClassifier не укладывается во временные ограничения соревнования, поэтому будем использовать гиперпараметры по умолчанию. Для LGBMClassifier параметры будут подобраны с методом hyperopt. При этом, в целях экономии времени, подбираться параметры будут на 20% от обучающей выборки.

Задаем пространство значений гиперпараметров:

In [41]:
space_lgbm = {
    'colsample_bynode' : hp.uniform('colsample_bynode', 0.5, 0.9),
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.1),
    'max_depth': hp.quniform('max_depth', 6, 10, 1),
    'subsample': hp.uniform('subsample', 0.5, 1),
    'min_child_weight': hp.uniform('min_child_weight', 0.1, 10),
    'reg_lambda': hp.uniform('reg_lambda', 0.1, 10),
    'reg_alpha': hp.uniform('reg_alpha', 0.1, 10),
    'num_leaves': hp.quniform('num_leaves', 8, 128, 1),
}

Подготавливаем 20%-ю подвыборку из обучающей выборки:

In [42]:
df = df_train.copy(deep=True)
df['WEEK_NUM'] = weeks
sample_df, sample_df_drop, sample_y, sample_y_drop = train_test_split(
    df,
    y,
    stratify=df['WEEK_NUM'],
    train_size=0.20,
    random_state=RS)
sample_df = sample_df.drop(['WEEK_NUM'], axis=1)
sample_df[cat_cols] = sample_df[cat_cols].astype('category')

Задаем параметры кросс-валидации:

In [43]:
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=RS)   # параметры стратификации/кросс-валидации

Задаем метод подбора гиперпараметров:

In [44]:
def objective_lgbm(params):
  
    # преобразование гиперпараметров в соответствующие типы
    params['max_depth'] = int(params['max_depth'])
    params['num_leaves'] = int(params['num_leaves'])
    
    # создание LGBMClassifier с указанными гиперпараметрами
    model = lgb.LGBMClassifier(
        **params,
        boosting_type = 'gbdt',
        objective = 'binary',
        metric = 'auc',
        n_estimators = 3000,
        extra_trees = True,
        device = 'gpu',
        verbose = -1,
        random_state=RS)
    
    pipeline = make_pipeline(
        model)
    
    # кросс-валидация
    scores = cross_val_score(pipeline,
                             sample_df,
                             sample_y,
                             cv=skf,
                             scoring='roc_auc',
                             error_score='raise')
    
    # расчет среднего значения метрики
    mean_score = np.mean(scores)
    
    print('mean_score: ', mean_score)
    
    return {'loss': 1 - mean_score, 'status': STATUS_OK}

Подбираем гиперпараметры:

In [45]:
# инициализация объекта trials
trials_lgbm = Trials()

# Запуск алгоритма подбора гиперпараметров
best_hyperparams_lgbm = fmin(fn=objective_lgbm, 
                        space=space_lgbm, 
                        algo=tpe.suggest, 
                        max_evals=200,
                        trials=trials_lgbm)

mean_score:                                            
0.7935701901235761                                     
mean_score:                                                                        
0.7959013730728872                                                                 
mean_score:                                                                         
0.804214806512001                                                                   
mean_score:                                                                         
0.8071594492729042                                                                  
mean_score:                                                                         
0.8046352751614447                                                                  
mean_score:                                                                         
0.8005633823189078                                                                  
mean_score:                             

Гиперпараметры max_depth и num_leaves приводим к целочисленному типу:

In [46]:
best_hyperparams_lgbm['max_depth'] = int(best_hyperparams_lgbm['max_depth'])
best_hyperparams_lgbm['num_leaves'] = int(best_hyperparams_lgbm['num_leaves'])

Результат подбора гиперпараметров:

In [47]:
for i, k in best_hyperparams_lgbm.items():
    print(i, k)

colsample_bynode 0.6064058213775632
learning_rate 0.02723950848121342
max_depth 10
min_child_weight 5.30230652055225
num_leaves 22
reg_alpha 9.980275920326584
reg_lambda 7.64028956336519
subsample 0.7178706784228773


[К содержанию](#start)

## Обучение модели
<a id="#section_5"></a>

Задаем параметры кросс-валидации:

In [48]:
cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=RS)

Обучаем модели:

In [49]:
%%time

fitted_models_cat = []
fitted_models_lgb = []

cv_scores_cat = []
cv_scores_lgb = []

# разделение выборки на обучающую и валидационную с группировкой по признаку WEEK_NUM
for idx_train, idx_valid in cv.split(df_train, y, groups=weeks):
    X_train, y_train = df_train.iloc[idx_train],y.iloc[idx_train]
    X_valid, y_valid = df_train.iloc[idx_valid], y.iloc[idx_valid]
    
    X_train = X_train.reset_index(drop=True)
    X_valid = X_valid.reset_index(drop=True)
    
    # обучение модели catboost
    train_pool = Pool(X_train, y_train,cat_features=cat_cols)
    val_pool = Pool(X_valid, y_valid,cat_features=cat_cols)
    clf = CatBoostClassifier(
    eval_metric='AUC',
    task_type='GPU',
    # learning_rate=0.03,
    iterations=1000,
    random_state=RS)
    clf.fit(train_pool, eval_set=val_pool,verbose=300)
    fitted_models_cat.append(clf)
    y_pred_valid = clf.predict_proba(X_valid)[:,1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores_cat.append(auc_score)



    # обучение модели LGBMClassifier
    X_train[cat_cols] = X_train[cat_cols].astype("category")
    X_valid[cat_cols] = X_valid[cat_cols].astype("category")
    model = lgb.LGBMClassifier(**best_hyperparams_lgbm,
                               boosting_type = 'gbdt',
                                objective = 'binary',
                                metric = 'auc',
                                n_estimators = 3000,
                                extra_trees = True,
                                device = 'gpu',
                                verbose = -1,
                               seed=RS)
    
    model.fit(
        X_train, y_train,
        eval_set = [(X_valid, y_valid)],
        callbacks = [lgb.log_evaluation(200), lgb.early_stopping(100)] )
    
    fitted_models_lgb.append(model)
    y_pred_valid = model.predict_proba(X_valid)[:,1]
    auc_score = roc_auc_score(y_valid, y_pred_valid)
    cv_scores_lgb.append(auc_score)
    
    
print('Значения ROC-AUC: {}'.format(cv_scores_cat))
print('Максимальное значение ROC-AUC: {}'.format(max(cv_scores_cat)))


print('Значения ROC-AUC: {}'.format(cv_scores_lgb))
print('Максимальное значение ROC-AUC: {}'.format(max(cv_scores_lgb)))

Learning rate set to 0.049211


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6738892	best: 0.6738892 (0)	total: 159ms	remaining: 2m 38s
300:	test: 0.8080694	best: 0.8080694 (300)	total: 2m 22s	remaining: 5m 30s
600:	test: 0.8136439	best: 0.8136439 (600)	total: 3m 56s	remaining: 2m 37s
900:	test: 0.8163020	best: 0.8163020 (900)	total: 5m	remaining: 33s
999:	test: 0.8171448	best: 0.8173051 (950)	total: 5m 23s	remaining: 0us
bestTest = 0.8173051476
bestIteration = 950
Shrink model to first 951 iterations.
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.806855
[400]	valid_0's auc: 0.818001
[600]	valid_0's auc: 0.821474
[800]	valid_0's auc: 0.822386
[1000]	valid_0's auc: 0.823008
Early stopping, best iteration is:
[1007]	valid_0's auc: 0.823126
Learning rate set to 0.049307


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6829605	best: 0.6829605 (0)	total: 225ms	remaining: 3m 44s
300:	test: 0.8207096	best: 0.8207096 (300)	total: 31.4s	remaining: 1m 12s
600:	test: 0.8253355	best: 0.8253475 (555)	total: 1m 2s	remaining: 41.2s
900:	test: 0.8264244	best: 0.8264605 (890)	total: 1m 32s	remaining: 10.2s
999:	test: 0.8270983	best: 0.8270983 (999)	total: 1m 43s	remaining: 0us
bestTest = 0.8270983398
bestIteration = 999
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.820419
[400]	valid_0's auc: 0.828626
[600]	valid_0's auc: 0.82989
Early stopping, best iteration is:
[643]	valid_0's auc: 0.830128
Learning rate set to 0.049012


...

0:	test: 0.6864943	best: 0.6864943 (0)	total: 1.42s	remaining: 23m 34s
300:	test: 0.8214554	best: 0.8214554 (300)	total: 1m 9s	remaining: 2m 41s
600:	test: 0.8258816	best: 0.8259363 (595)	total: 2m 18s	remaining: 1m 32s
900:	test: 0.8286920	best: 0.8287841 (895)	total: 3m	remaining: 19.8s
999:	test: 0.8297274	best: 0.8299409 (990)	total: 3m 10s	remaining: 0us
bestTest = 0.8299408853
bestIteration = 990
Shrink model to first 991 iterations.
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.81862
[400]	valid_0's auc: 0.827135
[600]	valid_0's auc: 0.830519
[800]	valid_0's auc: 0.831549
[1000]	valid_0's auc: 0.83263
[1200]	valid_0's auc: 0.832766
Early stopping, best iteration is:
[1114]	valid_0's auc: 0.832891
Learning rate set to 0.049147


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6895542	best: 0.6895542 (0)	total: 304ms	remaining: 5m 3s
300:	test: 0.8240511	best: 0.8240511 (300)	total: 31.4s	remaining: 1m 13s
600:	test: 0.8266777	best: 0.8266900 (565)	total: 1m 3s	remaining: 42s
900:	test: 0.8282424	best: 0.8283554 (865)	total: 1m 34s	remaining: 10.3s
999:	test: 0.8278233	best: 0.8283554 (865)	total: 1m 44s	remaining: 0us
bestTest = 0.8283554316
bestIteration = 865
Shrink model to first 866 iterations.
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.824063
[400]	valid_0's auc: 0.829737
[600]	valid_0's auc: 0.83059
Early stopping, best iteration is:
[514]	valid_0's auc: 0.830708
Learning rate set to 0.049015


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.6758666	best: 0.6758666 (0)	total: 362ms	remaining: 6m 2s
300:	test: 0.8159730	best: 0.8159730 (300)	total: 32s	remaining: 1m 14s
600:	test: 0.8219531	best: 0.8219531 (600)	total: 1m 3s	remaining: 42s
900:	test: 0.8241993	best: 0.8242862 (880)	total: 1m 35s	remaining: 10.5s
999:	test: 0.8248239	best: 0.8248239 (999)	total: 1m 45s	remaining: 0us
bestTest = 0.824823916
bestIteration = 999
Training until validation scores don't improve for 100 rounds
[200]	valid_0's auc: 0.810647
[400]	valid_0's auc: 0.821917
[600]	valid_0's auc: 0.825653
[800]	valid_0's auc: 0.827465
Early stopping, best iteration is:
[851]	valid_0's auc: 0.827917
Значения ROC-AUC: [0.8173051292211265, 0.8270983034454346, 0.8299408154949589, 0.8283552691295272, 0.8248237693330366]
Максимальное значение ROC-AUC: 0.8299408154949589
начения ROC-AUC: [0.8231257602668934, 0.8301283569495286, 0.8328905739995649, 0.830708299295029, 0.8279172958112506]
Максимальное значение ROC-AUC: 0.8328905739995649
CPU times: total

[К содержанию](#start)

## Расчет метрик
<a id="#section_6"></a>

Рассчитываем stability metric:  
stability metric = mean(gini) + 88,0 ⋅ min(0, a) − 0,5 ⋅ std(residuals)

Показатель gini рассчитывается для прогнозов, соответствующих каждому WEEK_NUM:
gini = 2 ⋅ AUC − 1

Линейная регрессия, а ⋅ х + b, проходит через недельные показатели gini, и a falling_rate рассчитывается как min(0, а).

Изменчивость прогнозов рассчитывается путем взятия стандартного отклонения остатков из приведенной выше линейной регрессии с применением штрафа к изменчивости модели.

Рассчитываем среднее значение ROC-AUC для каждого наблюдения по всем моделям:

In [50]:
class VotingModel(BaseEstimator, RegressorMixin):
    
    def __init__(self, estimators):
        super().__init__()
        self.estimators = estimators
        
    def fit(self, X, y=None):
        return self
    
    def predict_proba(self, X):
        
        y_preds = [estimator.predict_proba(X) for estimator in self.estimators[:5]]
        
        X[cat_cols] = X[cat_cols].astype("category")
        y_preds += [estimator.predict_proba(X) for estimator in self.estimators[5:]]
        
        return np.mean(y_preds, axis=0)

model = VotingModel(fitted_models_cat+fitted_models_lgb)

In [51]:
y_train_pred = pd.Series(model.predict_proba(df_train)[:, 1], index=df_train.index)
y_test_pred = pd.Series(model.predict_proba(df_test)[:, 1], index=df_test.index)

Для расчёта итоговой метрики добавляем в выборки признаки WEEK_NUM, target, score:

In [52]:
df_train = df_train_sync.join(df_train, how='left', on='case_id')
df_train['score'] = y_train_pred.tolist()
df_test = df_test_sync.join(df_test, how='left', on='case_id')
df_test['score'] = y_test_pred.tolist()

In [53]:
def gini_stability(base, w_fallingrate=88.0, w_resstd=-0.5):
    '''функция расчета итоговой метрики'''

    # расчет gini
    gini_in_time = base.loc[:, ["WEEK_NUM", "target", "score"]]\
        .sort_values("WEEK_NUM")\
        .groupby("WEEK_NUM")[["target", "score"]]\
        .apply(lambda x: 2*roc_auc_score(x["target"], x["score"])-1).tolist()
    
    x = np.arange(len(gini_in_time))
    y = gini_in_time
    a, b = np.polyfit(x, y, 1)
    y_hat = a*x + b
    residuals = y - y_hat
    res_std = np.std(residuals)
    avg_gini = np.mean(gini_in_time)
    return avg_gini + w_fallingrate * min(0, a) + w_resstd * res_std

In [54]:
gini_stability_train = gini_stability(df_train)
gini_stability_test = gini_stability(df_test)
print('Stability metric на обучающей выборке: {:.4f}'.format(gini_stability_train))
print('Stability metric на тестовой выборке: {:.4f}'.format(gini_stability_test))

Stability metric на обучающей выборке: 0.7753
Stability metric на тестовой выборке: 0.6193


[К содержанию](#start)

## Submission
<a id="#section_7"></a>

Создаем файл submission для выгрузки в kaggle:

In [55]:
df_test_kaggle.index = df_test_kaggle_sync['case_id']
y_pred = pd.Series(model.predict_proba(df_test_kaggle)[:, 1], index=df_test_kaggle.index)
df_subm = pd.read_csv(ROOT / "sample_submission.csv")
df_subm = df_subm.set_index("case_id")
df_subm["score"] = y_pred
df_subm.to_csv("submission.csv")
df_subm

,score
case_id,
57543,0.004946
57549,0.033609
57551,0.004570
57552,0.015644
57569,0.105471
57630,0.011335
57631,0.019822
57632,0.008067
57633,0.030141


## Общий вывод
<a id="#section_8"></a>

1. Получены 3 выборки:
   - обучающая - df_train;
   - тестовая - df_test;
   - тестовая для выгрузки в kaggle -df_test_kaggle.

2. Выполнена следующая предобработка:
   - получение агрегированных признаков (min, max, mean, first, last) для каждого значения case_id;
   - заполнение пропусков нечисловых признаков значением 'unknown';
   - преобразование типов данных;
   - генерация новых признаков из дат;
   - фильтрация признаков по порогу количества пропущенных значений и количеству уникальных значений;
   - фильтрация признаков по пороговому значению корреляции.

3. По результатам предобработки выборки имеют следующее количество признаков:
   - количество категориальных признаков: 38;
   - количество числовых признаков: 137;
   - общее количество признаков: 175.

4. Подобраны гиперпараметры модели LGBMClassifier.
  
5. Обучены модели CatBoostClassifier (с гиперпараметрами по умолчанию) и LGBMClassifier.

6. Значения stability metric на:
   - обучающей выборке: 0.7674;
   - тестовой выборке: 0.6131.

[К содержанию](#start)